In [1]:
import { pipeline, env } from "https://cdn.jsdelivr.net/npm/@xenova/transformers";
import { QdrantClient } from "npm:@qdrant/qdrant-js";

env.backends.onnx.wasm.numThreads = 1;

1

In [2]:
const embedder = await pipeline("feature-extraction", "Xenova/all-MiniLM-L6-v2")

In [68]:
const embedderOptions = { pooling: "mean", normalize: true}

In [5]:
const client = new QdrantClient({ host: "localhost", port: 6333})

In [6]:
const issues = JSON.parse(Deno.readTextFileSync("api/issues-cache.json"))

In [14]:
const vectors = []

In [16]:
for (const issue of issues) {
    const em = await embedder(issue.title + ": " + issue.body, {
      pooling: "mean",
      normalize: true,
    });
    vectors.push({
      id: issue.id,
      vector: em.tolist()[0],
      payload: {
        issue: {
          url: issue.html_url,
          title: issue.title,
          body: issue.body,
        },
      },
    });
}

2447

In [67]:
const logs = await (await fetch("http://localhost:8788/v0/logs")).json();
const jsxSampleError = logs.logs[1].message.stack

In [74]:
vectors[0].vector.length

384

In [86]:
const jsxSampleErrorEmbedding = (await embedder(jsxSampleError, embedderOptions)).tolist().flat()

# Now we're trying it out with Orama in-memory/disk persistence

In [59]:
import { create, insertMultiple, search } from 'npm:@orama/orama'

In [75]:
const db = await create({
  schema: {
    body: 'string',        // To make it simple, let's pretend that
    embedding: 'vector[384]', // we are using a 5-dimensional vector.
  }
})

In [83]:
const dataset = vectors.map(v=>{
    return {
        body: v.payload.issue.title + ": " + v.payload.issue.body ,
        embedding: v.vector
    }
});

In [85]:
await insertMultiple(db, dataset)

[
  "95064623-1",  "95064623-2",  "95064623-3",  "95064623-4",  "95064623-5",
  "95064623-6",  "95064623-7",  "95064623-8",  "95064623-9",  "95064623-10",
  "95064623-11", "95064623-12", "95064623-13", "95064623-14", "95064623-15",
  "95064623-16", "95064623-17", "95064623-18", "95064623-19", "95064623-20",
  "95064623-21", "95064623-22", "95064623-23", "95064623-24", "95064623-25",
  "95064623-26", "95064623-27", "95064623-28", "95064623-29", "95064623-30",
  "95064623-31", "95064623-32", "95064623-33", "95064623-34", "95064623-35",
  "95064623-36", "95064623-37", "95064623-38", "95064623-39", "95064623-40",
  "95064623-41", "95064623-42", "95064623-43", "95064623-44", "95064623-45",
  "95064623-46", "95064623-47", "95064623-48", "95064623-49", "95064623-50",
  "95064623-51", "95064623-52", "95064623-53", "95064623-54", "95064623-55",
  "95064623-56", "95064623-57", "95064623-58", "95064623-59", "95064623-60",
  "95064623-61", "95064623-62", "95064623-63", "95064623-64", "95064623-65"

In [89]:
const results = await search(db, {
  mode: 'vector',
  vector: {
    value: jsxSampleErrorEmbedding,
    property: 'embedding',
  },
  similarity: 0.6,      // Minimum similarity. Defaults to `0.8`
  includeVectors: true,  // Defaults to `false`
  limit: 33,             // Defaults to `10`
  offset: 0,             // Defaults to `0`
})

In [90]:
results

{
  count: 3,
  hits: [
    {
      id: "95064623-445",
      score: 0.6830262192378667,
      document: {
        body: "html renderer `isEscaped` TypeError: ### What version of Hono are you using?\n" +
          "\n" +
          "4.0.1\n" +
          "\n" +
          "### What runtim"... 1831 more characters,
        embedding: [
          -0.02282983809709549,
          -0.02159351482987404,
          0.0423736609518528,
          -0.03101913444697857,
          -0.04926181584596634,
          -0.05285801738500595,
          -0.03062395565211773,
          0.03202451020479202,
          -0.0281803198158741,
          0.011611485853791237,
          -0.00021630954870488495,
          -0.060184579342603683,
          -0.06411124765872955,
          0.00017331457638647407,
          0.00990719348192215,
          0.05118623003363609,
          0.052135664969682693,
          -0.0274581927806139,
          -0.0022632754407823086,
          0.0123941944912076,
          0.041142083704471